# Data Extraction

In [1]:
import fitz  # PyMuPDF
import re
from unstructured.partition.pdf import partition_pdf
import unstructured


class PDFProcessor:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.full_text = ""
        self.chapters = []
        self.section_texts = {}

    def extract_and_clean_text(self):
        doc = fitz.open(self.pdf_path)

        for page in doc:
            text = self._extract_text_from_page(page)
            cleaned_text = self._clean_extracted_text(text)
            self.full_text += cleaned_text

        return self.full_text

    def _extract_text_from_page(self, page):
        text = ""
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    for span in line["spans"]:
                        is_bold = span['flags'] & 16  # Bold flag
                        is_italic = span['flags'] & 2  # Italic flag
                        if span['size'] >= 14.0:  # Title
                            text += "# " + span['text'] + "\n"
                        else:
                            text += span['text'] + "\n"
        return text

    @staticmethod
    def _clean_extracted_text(text):
        lines = text.split('\n')
        cleaned_lines = []
        last_line_was_heading = False

        for line in lines:
            line = line.strip()
            if not line:
                continue

            current_line_is_heading = line.startswith("#")
            if current_line_is_heading and last_line_was_heading:
                line = line.replace('#', '')
            last_line_was_heading = current_line_is_heading

            if re.match(r'^#\s+CHAPITRE', line):
                line = "\n\n" + line

            if line.endswith('.'):
                cleaned_lines.append(line + "\n")
            else:
                cleaned_lines.append(line + " ")

        return ''.join(cleaned_lines).strip()

    def partition_pdf(self):
        elements = partition_pdf(self.pdf_path, strategy="hi_res")
        self.chapters = [
            element.text for element in elements if isinstance(element, unstructured.documents.elements.Title)
        ]

    def split_text_by_chapters(self):
        pattern = '|'.join([re.escape(section) for section in self.chapters])
        split_text = re.split(pattern, self.full_text)
        split_text = [section for section in split_text if section.strip()]

        for i, section in enumerate(self.chapters):
            if i < len(split_text):
                self.section_texts[section] = split_text[i + 1].strip()

    def transform_sections_to_array(self):
        structured_text = []
        for section, content in self.section_texts.items():
            structured_text.append(f"Chapter: {section}\nContent: {content}")
        return structured_text

    @staticmethod
    def transform_array(array):
        transformed_array = []

        for entry in array:
            chapter_title_match = re.match(r'Chapter: ([^\n]+)', entry)
            if not chapter_title_match:
                continue
            chapter_title = chapter_title_match.group(1)

            content_match = re.search(r'Content: (.+)', entry, re.DOTALL)
            if not content_match:
                continue
            content = content_match.group(1)

            sentences = re.split(r'(?<=\.) ', content)
            for sentence in sentences:
                transformed_array.append(f'Chapter: {chapter_title}\n{sentence.strip()}')

        return transformed_array


# Main execution
if __name__ == "__main__":
    pdf_path = "hackaton-mistral-studai/data/RAGAS_09_2023.pdf"
    processor = PDFProcessor(pdf_path)

    text = processor.extract_and_clean_text()
    processor.partition_pdf()
    processor.split_text_by_chapters()
    structured_text = processor.transform_sections_to_array()
    transformed_text = PDFProcessor.transform_array(structured_text)

    for line in transformed_text:
        print(line)
        print()


Chapter: RAGAS: Automated Evaluation of Retrieval Augmented Generation
Shahul Es † , Jithin James † , Luis Espinosa-Anke ∗♢ , Steven Schockaert ∗ † Exploding Gradients ∗ CardiffNLP, Cardiff University, United Kingdom ♢ AMPLYFI, United Kingdom shahules786@gmail.com,jamesjithin97@gmail.com {espinosa-ankel,schockaerts1}@cardiff.ac.uk

Chapter: Abstract
We introduce RAGA S ( R etrieval A ugmented G eneration As sessment), a framework for reference-free evaluation of Retrieval Aug- mented Generation (RAG) pipelines.
RAG systems are composed of a retrieval and an LLM based generation module, and provide LLMs with knowledge from a reference textual database, which enables them to act as a natu- ral language layer between a user and textual databases, reducing the risk of hallucinations.
Evaluating RAG architectures is, however, chal- lenging because there are several dimensions to consider: the ability of the retrieval system to identify relevant and focused context passages, the ability of t

In [1]:
import fitz  # PyMuPDF
import re

def clean_extracted_text(text):
    import re  # Import regular expressions library
    lines = text.split('\n')
    cleaned_lines = []
    last_line_was_heading = False  # To track consecutive heading lines

    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        # Handle consecutive heading lines
        current_line_is_heading = line.startswith("#")
        if current_line_is_heading and last_line_was_heading:
           line = line.replace('#', '')  # Remove all '#' symbols from the line
        last_line_was_heading = current_line_is_heading

        # Use a regular expression to match lines starting with '#' followed by one or several spaces and 'CHAPITRE'
        if re.match(r'^#\s+CHAPITRE', line):
            line = "\n\n" + line  # Prepend two line breaks

        # Decide on line breaks based on punctuation and specific cases
        if line.endswith('.'):
            # End of sentence or apostrophe, allow for normal line break
            cleaned_lines.append(line + "\n")
        else:
            # No line break for continuing sentences or lines ending with "l'"
            cleaned_lines.append(line + " ")

    return ''.join(cleaned_lines).strip()


def extract_and_clean_text(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""

    for page in doc:
        text = ""
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Check for bold and italic
                        is_bold = span['flags'] & 16  # Bold flag
                        is_italic = span['flags'] & 2  # Italic flag
                        # if span['size'] == 11.0 and is_bold and is_italic:
                        #     text += "## " + span['text'] + "\n"  # Consider as subtitle
                        if span['size'] >= 14.0:  # Title
                            text += "# " + span['text'] + "\n"
                        # elif span['size'] >= 11.0:  # Subtitle
                        #     text += "## " + span['text'] + "\n"
                        else:
                            text += span['text'] + "\n"
        # Clean the extracted text before adding it to the full_text
        cleaned_text = clean_extracted_text(text)
        full_text += cleaned_text

    return full_text


# Specify the path to your PDF here
pdf_path = "hackaton-mistral-studai/data/RAGAS_09_2023.pdf"
text = extract_and_clean_text(pdf_path)
print(text)


# RAGAS: Automated Evaluation of Retrieval Augmented Generation Shahul Es † , Jithin James † , Luis Espinosa-Anke ∗♢ , Steven Schockaert ∗ † Exploding Gradients ∗ CardiffNLP, Cardiff University, United Kingdom ♢ AMPLYFI, United Kingdom shahules786@gmail.com,jamesjithin97@gmail.com {espinosa-ankel,schockaerts1}@cardiff.ac.uk Abstract We introduce RAGA S ( R etrieval A ugmented G eneration As sessment), a framework for reference-free evaluation of Retrieval Aug- mented Generation (RAG) pipelines.
RAG systems are composed of a retrieval and an LLM based generation module, and provide LLMs with knowledge from a reference textual database, which enables them to act as a natu- ral language layer between a user and textual databases, reducing the risk of hallucinations.
Evaluating RAG architectures is, however, chal- lenging because there are several dimensions to consider: the ability of the retrieval system to identify relevant and focused context passages, the ability of the LLM to exploit

In [2]:
from unstructured.partition.pdf import partition_pdf
from collections import Counter

# unstructured = os.path.join("hackaton-mistral-studai/data/", "RAGAS_09_2023.pdf")
elements = partition_pdf("hackaton-mistral-studai/data/RAGAS_09_2023.pdf", strategy="hi_res")
display(Counter(type(element) for element in elements))

Counter({unstructured.documents.elements.NarrativeText: 59,
         unstructured.documents.elements.ListItem: 28,
         unstructured.documents.elements.Title: 12,
         unstructured.documents.elements.Text: 5,
         unstructured.documents.elements.Table: 4,
         unstructured.documents.elements.Formula: 2,
         unstructured.documents.elements.Header: 1})

In [3]:
import unstructured
# for i in [(type(element), element.text) for element in elements[:] if type(element)==unstructured.documents.elements.Title]:
#     print(i)

# Create tuples (type(element), element.text) ensuring the elements are Title objects
elements_tuples = [(type(element), element.text) for element in elements if isinstance(element, unstructured.documents.elements.Title)]

# Extract and print the second column of each tuple
chapters = [element[1] for element in elements_tuples]
chapters

['RAGAS: Automated Evaluation of Retrieval Augmented Generation',
 'Abstract',
 'Introduction',
 '2 Related Work',
 '3 Evaluation Strategies',
 '4 The WikiEval Dataset',
 '5 Experiments',
 '6 Conclusions',
 'References',
 'A Examples from WikiEval',
 'Question',
 'Context']

In [4]:
# Create a regular expression pattern to match the section titles
pattern = '|'.join([re.escape(section) for section in chapters])
split_text = re.split(pattern, text)

# The split_text list will contain empty strings at the start and end, and the text of sections in between
# Remove empty strings from the list
split_text = [section for section in split_text if section.strip()]

# Create a dictionary with section titles as keys and section texts as values
section_texts = {}
for i, section in enumerate(chapters):
    if i < len(split_text):
        section_texts[section] = split_text[i+1].strip()

structured_text = []
# Print the sections and their corresponding texts
for section, content in section_texts.items():
    structured_text.append(f"Chapter: {section}\nContent: {content}")
    # print(f"Section: {section}\nContent: {content}\n{'-'*40}")

structured_text

['Chapter: RAGAS: Automated Evaluation of Retrieval Augmented Generation\nContent: Shahul Es † , Jithin James † , Luis Espinosa-Anke ∗♢ , Steven Schockaert ∗ † Exploding Gradients ∗ CardiffNLP, Cardiff University, United Kingdom ♢ AMPLYFI, United Kingdom shahules786@gmail.com,jamesjithin97@gmail.com {espinosa-ankel,schockaerts1}@cardiff.ac.uk',
 'Chapter: Abstract\nContent: We introduce RAGA S ( R etrieval A ugmented G eneration As sessment), a framework for reference-free evaluation of Retrieval Aug- mented Generation (RAG) pipelines.\nRAG systems are composed of a retrieval and an LLM based generation module, and provide LLMs with knowledge from a reference textual database, which enables them to act as a natu- ral language layer between a user and textual databases, reducing the risk of hallucinations.\nEvaluating RAG architectures is, however, chal- lenging because there are several dimensions to consider: the ability of the retrieval system to identify relevant and focused context

In [5]:
import re

# Function to transform the array
def transform_array(array):
    transformed_array = []
    
    for entry in array:
        # Extract chapter title
        chapter_title_match = re.match(r'Chapter: ([^\n]+)', entry)
        if not chapter_title_match:
            continue
        chapter_title = chapter_title_match.group(1)
        
        # Extract content
        content_match = re.search(r'Content: (.+)', entry, re.DOTALL)
        if not content_match:
            continue
        content = content_match.group(1)
        
        # Split content into sentences
        sentences = re.split(r'(?<=\.) ', content)
        
        # Prepend chapter title to each sentence and add to transformed array
        for sentence in sentences:
            transformed_array.append(f'Chapter: {chapter_title}\n{sentence.strip()}')
    
    return transformed_array

# Transform the array
structured_text = transform_array(structured_text)

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_core.documents import Document

# Convert list to string with a space separator
structured_text = ' '.join(structured_text)

# splitter = CharacterTextSplitter(separator="Chapter:", chunk_size=30, chunk_overlap=10)
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20, separators=['Chapter:','\n\n'])
chunks = splitter.split_text(structured_text)
docs = splitter.create_documents(chunks)
docs

[Document(page_content='Chapter: RAGAS: Automated Evaluation of Retrieval Augmented Generation\nShahul Es † , Jithin James † , Luis Espinosa-Anke ∗♢ , Steven Schockaert ∗ † Exploding Gradients ∗ CardiffNLP, Cardiff University, United Kingdom ♢ AMPLYFI, United Kingdom shahules786@gmail.com,jamesjithin97@gmail.com {espinosa-ankel,schockaerts1}@cardiff.ac.uk '),
 Document(page_content='Chapter: Abstract\nWe introduce RAGA S ( R etrieval A ugmented G eneration As sessment), a framework for reference-free evaluation of Retrieval Aug- mented Generation (RAG) pipelines.\nRAG systems are composed of a retrieval and an LLM based generation module, and provide LLMs with knowledge from a reference textual database, which enables them to act as a natu- ral language layer between a user and textual databases, reducing the risk of hallucinations.\nEvaluating RAG architectures is, however, chal- lenging because there are several dimensions to consider: the ability of the retrieval system to identify 

# Mistral

In [10]:
from mistralai.client import MistralClient

api_key = "ImsUHfFLA6OjlX6mARbnM1YcDOy7ujsq"
client = MistralClient(api_key="ImsUHfFLA6OjlX6mARbnM1YcDOy7ujsq")

## Embedding

### Manualy

In [6]:
def get_text_embedding(txt):
    client = MistralClient(api_key=api_key)
    embeddings_batch_response = client.embeddings(model="mistral-embed", input=txt)
    return embeddings_batch_response.data[0].embedding

In [7]:
import numpy as np

text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
text_embeddings

NameError: name 'get_text_embedding' is not defined

### Langchain + Chroma

In [12]:
from langchain_mistralai import MistralAIEmbeddings
from langchain.vectorstores import Chroma

output_path = "hackaton-mistral-studai/data/chromadb"
mistral_embeddings = MistralAIEmbeddings(api_key=api_key)
mistral_embeddings.model = "mistral-embed"  
chroma_db = Chroma.from_documents(docs, mistral_embeddings, persist_directory=output_path)

### Faiss

In [8]:
""" Vector Database """
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

## RAG

### Manualy

In [9]:
question = "What is RAGAs?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings

array([[-0.00650787, -0.04208374,  0.06274414, ..., -0.02349854,
         0.03463745,  0.02111816]])

In [10]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[18 20]]


In [11]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['\nTo address these issues, in this paper we present RAGA S 1 , a framework for the automated assess- 1 RAGA S is available at https://github.com/ explodinggradients/ragas .', '\nWe focus on settings where reference answers may not be available, and where we want to estimate different proxies for correctness, in addition to the usefulness of the retrieved passages. The RAGA S framework provides an integration with both llama- index and Langchain , the most widely used frame- works for building RAG solutions, thus enabling developers to easily integrate RAGA S into their standard workflow.']


In [12]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

from mistralai.models.chat_completion import ChatMessage

client = MistralClient(api_key="ImsUHfFLA6OjlX6mARbnM1YcDOy7ujsq")

def mistral(user_message, model="mistral-small-latest", is_json=False):
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [13]:
response = mistral(prompt)
print(response)

RAGA S is a framework for automated assessment, specifically designed for settings where reference answers may not be available. It aims to estimate different proxies for correctness and the usefulness of retrieved passages. The RAGA S framework is available for integration with llama-index and Langchain, which are widely used frameworks for building RAG (Retrieval-Augmented Generation) solutions. You can find RAGA S at this GitHub link: https://github.com/explodinggradients/ragas.


### Langchain + Chroma

In [13]:
# Load the vector database to confirm it contains the data
path = "hackaton-mistral-studai/data/chromadb"
db = Chroma(persist_directory=path)

# Optionally, verify some details about the stored vectors
print("Number of vectors stored:", len(db))


Number of vectors stored: 167


In [6]:
from langchain_mistralai import MistralAIEmbeddings
from langchain.vectorstores import Chroma
from mistralai.client import MistralClient

path = "hackaton-mistral-studai/data/chromadb"

api_key = "ImsUHfFLA6OjlX6mARbnM1YcDOy7ujsq"
client = MistralClient(api_key="ImsUHfFLA6OjlX6mARbnM1YcDOy7ujsq")

# Mistral Embedding model
mistral_embeddings = MistralAIEmbeddings(api_key=api_key)
mistral_embeddings.model = "mistral-embed"  

# load vector_db
db = Chroma(persist_directory=path, embedding_function=mistral_embeddings)

# define retriever
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 15})
retriever.invoke("Chapter: Introduction")

/home/maximejabarian/miniconda3/envs/studai_env/lib/python3.11/site-packages/langchain_mistralai/embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


[Document(page_content='Chapter: Introduction\nChatGPT and GPT-4).'),
 Document(page_content='Chapter: Introduction\nby mea- suring perplexity on some reference corpus.'),
 Document(page_content='Chapter: Introduction\nAutomated evaluation of retrieval-augmented systems is thus paramount.'),
 Document(page_content='Chapter: Introduction\nIn practice, RAG systems are often evaluated in terms of the language modelling task itself, i.e. '),
 Document(page_content='Chapter: Introduction\nWhile initial approaches relied on specialised LMs for retrieval-augmented language modelling ( Khandel- wal et al.\n, 2020 ; Borgeaud et al.\n, 2022 ), recent work has suggested that simply adding retrieved docu- ments to the input of a standard LM can also work well ( Khattab et al.\n, 2022 ; Ram et al.\n, 2023 ; Shi et al.\n, 2023 ), thus making it possible to use retrieval- augmented strategies in combination with LLMs that are only available through APIs.\nWhile the usefulness of retrieval-augmented s

In [38]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Define LLM
model = ChatMistralAI(mistral_api_key=api_key, model_name="open-mixtral-8x22b")
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "To check if I really understood what's RAGAS, give me short explanation whith missing words to complete with a list of words to fill with, as if I was a child of 10 years old."})
print(response["answer"])

RAGAS is a special tool that helps us check how good a computer program is at answering questions using information it finds. It's like asking your friend to find the answer to a question in a book, but instead of a friend, it's a computer program. The words to fill in are: computer program, answers, information, book.

So, RAGAS helps us check if a computer program can find the right answers using information it finds, just like your friend finding answers in a book.


## RAG Chatbox

In [ ]:
import os
from mistralai.models.chat_completion import ChatMessage
from mistralai.client import MistralClient
import panel as pn
pn.extension()

In [15]:
def run_mistral(contents, user, chat_interface):
    messages = [ChatMessage(role="user", content=contents)]
    chat_response = client.chat(
        model="mistral-large-latest", 
        messages=messages)
    return chat_response.choices[0].message.content

In [16]:
chat_interface = pn.chat.ChatInterface(
    callback=run_mistral, 
    callback_user="Mistral"
)

chat_interface

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<function run_mistral a..., callback_user='Mistral', show_button_name=True, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])

In [17]:
import numpy as np
import faiss

client = MistralClient(api_key="ImsUHfFLA6OjlX6mARbnM1YcDOy7ujsq")

prompt = """
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

def get_text_embedding(input):
    embeddings_batch_response = client.embeddings(model="mistral-embed", input=input)
    return embeddings_batch_response.data[0].embedding

def run_mistral(user_message, model="mistral-large-latest"):
    messages = [ChatMessage(role="user", content=user_message)]
    chat_response = client.chat(model=model, messages=messages)
    return chat_response.choices[0].message.content

def answer_question(question, index):
    # create embeddings for a question
    question_embeddings = np.array([get_text_embedding(question)])
    # retrieve similar chunks from the vector database
    D, I = index.search(question_embeddings, k=2)
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    # generate response based on the retrieved relevant text chunks
    response = run_mistral(
        prompt.format(retrieved_chunk=retrieved_chunk, question=question)
    )

    return response

In [18]:
chat_interface = pn.chat.ChatInterface(
    callback=answer_question,
    callback_user="Mistral",
)
chat_interface.send(
    "Send a message to get a reply from Mistral!", 
    user="System", 
    respond=False
)
chat_interface

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<function answer_question ..., callback_user='Mistral', show_button_name=True, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])
    [0] ChatMessage(str, avatar='⚙️', reaction_icons=ChatReactionIcons, timestamp=datetime.datetime(2024, ..., user='System')

In [19]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import param
from mistralai.models.chat_completion import ChatMessage

# Replace with your actual API key
client = MistralClient(api_key="ImsUHfFLA6OjlX6mARbnM1YcDOy7ujsq")

def get_text_embedding(txt):
    client = MistralClient(api_key=api_key)
    embeddings_batch_response = client.embeddings(model="mistral-embed", input=txt)
    return embeddings_batch_response.data[0].embedding

class cbfs(param.Parameterized):
    chat_history = param.List(default=[])
    answer = param.String(default="")
    
    def __init__(self, **params):
        super(cbfs, self).__init__(**params)
    
    def convchain(self, query):
        if not query:
            return
        question_embeddings = np.array([get_text_embedding(query)])
        D, I = index.search(question_embeddings, k=2)
        retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

        prompt = f"""
        Context information is below.
        ---------------------
        {retrieved_chunk}
        ---------------------
        Given the context information and not prior knowledge, answer the query.
        Query: {query}
        Answer:
        """

        messages = [ChatMessage(role="user", content=prompt)]
        chat_response = client.chat(model="mistral-small-latest", messages=messages)
        answer = chat_response.choices[0].message.content

        self.chat_history.append((query, answer))
        self.answer = answer

In [20]:
class ChatInterface:
    def __init__(self, cbfs_instance):
        self.cbfs_instance = cbfs_instance
        self.user_input = widgets.Text(
            placeholder='Type your question here...',
            description='You:',
            disabled=False
        )
        self.send_button = widgets.Button(description='Send')
        self.chat_output = widgets.Output()

        self.send_button.on_click(self.on_send)
        self.user_input.on_submit(self.on_send_enter)  # Use on_submit for the enter key event
        display(widgets.VBox([self.user_input, self.send_button, self.chat_output]))

    def on_send(self, b):
        self.process_user_input()

    def on_send_enter(self, widget):  # Handler for the Enter key
        self.process_user_input()

    def process_user_input(self):
        user_query = self.user_input.value.strip()
        if user_query:
            self.cbfs_instance.convchain(user_query)
            with self.chat_output:
                self.display_chat_history()
            self.user_input.value = ''  # Clear input after sending

    def display_chat_history(self):
        self.chat_output.clear_output()
        with self.chat_output:
            for query, response in self.cbfs_instance.chat_history:
                print(f'User: {query}')
                print(f'AI: {response}')

In [21]:
# Initialize and display the chat interface with a cbfs instance
cb_instance = cbfs()
chat_interface = ChatInterface(cb_instance)

/tmp/ipykernel_30224/2273792851.py:13: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.user_input.on_submit(self.on_send_enter)  # Use on_submit for the enter key event


# Groq

In [22]:
import os
from groq import Groq

# Set the environment variable in the script
os.environ["GROQ_API_KEY"] = "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"

client = Groq(api_key=os.environ.get("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"))

for i in range(20):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Explain the importance of fast language models",
            }
        ],
        model="llama3-70b-8192",
    )
    print("-"*20)
    print(chat_completion.choices[0].message.content)

--------------------
Fast language models are a type of artificial intelligence (AI) that can process and understand human language quickly and efficiently. The importance of fast language models lies in their ability to enable rapid natural language processing (NLP) capabilities, which have numerous benefits in various applications. Here are some reasons why fast language models are important:

1. **Real-time interactions**: Fast language models enable real-time interactions between humans and machines, making them essential for applications like chatbots, virtual assistants, and speech recognition systems.
2. **Improved user experience**: Fast language models can quickly respond to user queries, providing a more seamless and responsive user experience in applications like customer service chatbots, voice assistants, and language translation apps.
3. **Increased productivity**: Fast language models can automate many NLP tasks, freeing up human time and effort for more creative and str